In [1]:
import torch
import os
from torchvision import datasets, models, transforms
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import glob
import copy
import neptune
#import splitfolders # Библиотека для разделения файлов картинок на train test
import matplotlib.gridspec as gridspec
cudnn.benchmark = True

In [16]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'C:\data\dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [ ]:
dataloaders

In [17]:
model_ft = models.resnet50()


num_ftrs = model_ft.fc.in_features


model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)


model_ft.load_state_dict(torch.load('weights-0.894737acc-0.2899loss.pth',map_location ='cpu'))


nb_classes = 10

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)


tensor([[ 60.,  12.,   3.,   2.,   1.,   1.,   5.,   0.,  15.,   0.],
        [  9.,  83.,   4.,   8.,   0.,   0.,  12.,   0.,  12.,   1.],
        [  4.,   8., 113.,   4.,   0.,   0.,   6.,   0.,  12.,   0.],
        [  0.,   1.,   2., 129.,   3.,   0.,   5.,   0.,  25.,   0.],
        [  0.,   2.,   0.,   5.,  43.,   0.,   8.,   0.,   6.,   0.],
        [  0.,   0.,   1.,   4.,   0.,  35.,   3.,   0.,   0.,   0.],
        [  4.,  13.,   5.,   4.,   0.,   1., 129.,   0.,  21.,   0.],
        [  0.,   4.,   0.,   0.,   0.,   0.,   0.,  20.,   2.,   2.],
        [  4.,   5.,   3.,  16.,   0.,   0.,  21.,   0., 113.,   0.],
        [  1.,   1.,   0.,   2.,   0.,   0.,   6.,   0.,   0.,  59.]])


In [18]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

tensor([0.6061, 0.6434, 0.7687, 0.7818, 0.6719, 0.8140, 0.7288, 0.7143, 0.6975,
        0.8551])


In [19]:
from sklearn.metrics import confusion_matrix

nb_classes = 10

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)

[[ 68   9   2   0   0   0   6   0  14   0]
 [  7  88   5   5   0   0  15   0   9   0]
 [  6   6 112   5   0   1   7   0  10   0]
 [  3   1   2 128   4   0   7   0  20   0]
 [  0   0   0   6  43   0   6   0   9   0]
 [  0   0   3   2   0  31   7   0   0   0]
 [  2   9   4   3   1   0 134   0  24   0]
 [  2   2   0   0   1   0   1  21   0   1]
 [  4   4   1  13   1   0  19   0 120   0]
 [  2   3   1   0   0   0   4   0   1  58]]
[68.68686869 68.21705426 76.19047619 77.57575758 67.1875     72.09302326
 75.70621469 75.         74.07407407 84.05797101]
